In [4]:
import csv

import numpy as np

try:
    from gensim import models
except ModuleNotFoundError as e:
    !pip install gensim==3.8.0
    from gensim import models
try:
    import pandas as pd
except ModuleNotFoundError as e:
    !pip install pandas
    import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
# This line prevents TF crashing when using convolutional networks
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
def prepare_data(model_path, data_prefix, seq_len, embedding_size):
    """
    
    Args:
        model_path (str): path to embedding model
        data_prefix (str): path to data prefix
    
    """
    
    def load_data(path):
        output = []
        
        with open(path, 'r', encoding='latin-1') as data:
        #with open(path, 'r', encoding='utf-8') as data:
            for row in csv.reader(data):
                output.append(row)
                
        return output
    
    # Opens embedding model
    model_ = models.Word2Vec.load(model_path)
    
    # Open dataset
    data_train = load_data(data_prefix + "X_train.csv")
    data_val = load_data(data_prefix + "X_val.csv")
    label_train = np.loadtxt(data_prefix + "y_train.csv")
    label_val = np.loadtxt(data_prefix + "y_val.csv")
    
    # Gets embeddings from model
    dt = []
    lt = []
    omissions_ = 0
    
    for i, seq in enumerate(data_train):
        
        try:        
            embedding = model_.wv[seq]
            dt.append(embedding)
            lt.append(label_train[i])
        
        except KeyError as ke:
            for word in seq:
                if word not in model_.wv.vocab.keys():
                    seq.remove(word)
            
        except ValueError as ve:
            omissions_ += 1
            
#     print(omissions_)
    
    # Gets embeddings from model
    dv = []
    lv = []
    omissions_ = 0
    
    for i, seq in enumerate(data_val):
        
        try:
            embedding = model_.wv[seq]
            dv.append(embedding)
            lv.append(label_val[i])
        
        except KeyError as ke:
            for word in seq:
                if word not in model_.wv.vocab.keys():
                    seq.remove(word)
            
        except ValueError as ve:
            omissions_ += 1
    
#     print(omissions_)
    
    # Pads sequences
    dt = pad_sequences(dt, padding='post', dtype='float64', maxlen=seq_len)
    dv = pad_sequences(dv, padding='post', dtype='float64', maxlen=seq_len)
    
    # Converts lists to numpy arrays
#     dt = np.asarray(dt).reshape((len(dt), seq_len * embedding_size))
#     dv = np.asarray(dv).reshape((len(dv), seq_len * embedding_size))
    
    lt = np.asarray(lt)
    lv = np.asarray(lv)
    
    return dt, dv, lt, lv

In [7]:
simpson_dict = {5: "./resources/embeddings/Simpsons_5_7.model",
                25: "./resources/embeddings/Simpsons_25_7.model",
                125: "./resources/embeddings/Simpsons_125_7.model",
                'prefix': "./data/simpsons/",
                'classes': 4}

friends_dict = {5: "./resources/embeddings/Friends_5_7.model",
                25: "./resources/embeddings/Friends_25_7.model",
                125: "./resources/embeddings/Friends_125_7.model",
                'prefix': "./data/friends/",
                'classes': 6}

In [8]:
def create_checkpoint_callback(filepath):
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = filepath,
        save_weights_only = False,
        monitor = 'accuracy',
        mode = 'max',
        save_best_only = True)
    return model_checkpoint_callback

## Model creators

In [9]:
def create_base_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.Dense((embedding_size * seq_len)/2, activation="relu", input_shape = input_shape),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [10]:
def create_deepFC_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.Dense((embedding_size * seq_len)/2, activation="relu", input_shape = input_shape),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [11]:
def create_RNN_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.SimpleRNN(units = 20, return_sequences=True, input_shape=input_shape),
        layers.SimpleRNN(units = 20, return_sequences=True),
        layers.SimpleRNN(units = 20, return_sequences=True),
        layers.Flatten(),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [12]:
def create_LSTM_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.LSTM(units = 20, return_sequences=True, input_shape = input_shape),
        layers.LSTM(units = 20, return_sequences=True),
        layers.LSTM(units = 20, return_sequences=True),
        layers.Flatten(),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [13]:
model_creators = {'baseline': create_base_model,
                  'DeepFC': create_deepFC_model,
                  'SimpleRNN': create_RNN_model,
                  'LSTM': create_LSTM_model}

## Training methods

In [28]:
def train_val_model(dataset, em_size, seq_len, model_type, epochs):
    if dataset == 'simpson':
        data_path = simpson_dict
    elif dataset == 'friends':
        data_path = friends_dict
    else: 
        raise 'Not valid dataset'
    X_train, X_val, y_train, y_val = prepare_data(data_path[em_size], data_path['prefix'],
                                                  seq_len, em_size)
    if model_type == 'baseline':
        X_train = np.asarray(X_train).reshape((len(X_train), seq_len * em_size))
        X_val = np.asarray(X_val).reshape((len(X_val), seq_len * em_size))
    callback = create_checkpoint_callback('./resources/checkpoints/' + model_type + '_' + 
                                          str(em_size) + '_' +str(seq_len))
    input_shape = (len(X_train), seq_len * em_size) if model_type in ['baseline','deepFC'] else (seq_len, em_size)
    model = model_creators[model_type](em_size, seq_len, data_path['classes'], 
                                       input_shape=input_shape)
    model.fit(X_train, y_train, epochs = epochs, verbose = 0, callbacks=[callback])
    model_metrics = model.evaluate(x = X_val, y = y_val)
    data = {'model_name': [model_type], 'embedding_size': [em_size], 'seq_len': [seq_len],
           'accuracy': [model_metrics[1]], 'precision': [model_metrics[2]], 'recall': [model_metrics[3]]}
    return pd.DataFrame(data=data)

## Model Training

In [23]:
SEQ_LEN_ = [8, 11, 16]
EM_SIZE_ = [5, 25, 125]
metrics_df = pd.DataFrame()
for seq_len in SEQ_LEN_:
    for em_size in EM_SIZE_:
        model_results = train_val_model('simpson', em_size, seq_len, 'baseline', 30)
        metrics_df = pd.concat([metrics_df, model_results])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 45297, 20)         820       
_________________________________________________________________
dense_23 (Dense)             (None, 45297, 4)          84        
Total params: 904
Trainable params: 904
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resour

INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_8/assets
INFO:tensorflow:Assets writt

INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
INFO:tensorflow:Assets writt

INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_16/assets
257/257 [==============================] - 1s 2ms/step - loss: 1.3377 - accuracy: 0.4179 - precision_17: 0.4824 - recall_17: 0.0968
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 45297, 200)        80200     
_______

INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_16/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_16/assets
257/257 [==============================] - 1s 2ms/step - loss: 4.1723 - accuracy: 0.4078 - precision_19: 0.4154 - recall_19: 0.3740


In [24]:
metrics_df

,model_name,embedding_size,seq_len,accuracy,precision,recall
0,baseline,5,8,0.435329,0.518349,0.055102
0,baseline,25,8,0.389248,0.421132,0.271120
0,baseline,125,8,0.375350,0.380038,0.342558
0,baseline,5,11,0.428989,0.474262,0.068512
0,baseline,25,11,0.367792,0.377158,0.298306
0,baseline,125,11,0.383884,0.390601,0.347556
0,baseline,5,16,0.417896,0.482382,0.096794
0,baseline,25,16,0.370596,0.380130,0.327441
0,baseline,125,16,0.407778,0.415380,0.374010


In [ ]:
MODEL_TYPES_ = ['SimpleRNN', 'LSTM']
for model_type in MODEL_TYPES_: 
    for em_size in EM_SIZE_:
        model_results = train_val_model('simpson', em_size, 16, model_type, 200)
        metrics_df = pd.concat([metrics_df, model_results])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_6 (SimpleRNN)     (None, 16, 20)            520       
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 16, 20)            820       
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 16, 20)            820       
_________________________________________________________________
flatten_2 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 40)                12840     
_________________________________________________________________
dense_45 (Dense)             (None, 4)                 164       
Total params: 15,164
Trainable params: 15,164
Non-trainable params: 0
_________________________________________________

## Fully conected

## Simple RNN

In [11]:
def create_RNN_model(embedding_size, seq_len, classes):
    model = keras.Sequential([
        layers.SimpleRNN(units = 20, return_sequences=True, input_shape=(seq_len, embedding_size)),
        layers.SimpleRNN(units = 20, return_sequences=True),
        layers.SimpleRNN(units = 20, return_sequences=True),
        layers.Flatten(),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [13]:
epochs = 300
embedding_size = 5
seq_len = 11
X_train, X_val, y_train, y_val = prepare_data(simpson_model_5_path, simpson_prefix, seq_len, embedding_size)
callback = create_checkpoint_callback('./resources/checkpoints/baseline_' + 
                                      str(embedding_size) + '_' +str(seq_len))
model = create_RNN_model(embedding_size, seq_len, 4)
model.fit(X_train, y_train, epochs = epochs, verbose=2, callbacks=[callback])
model_metrics = model.evaluate(x=X_val, y=y_val)
data = {'model_name': 'simpleRNN', 'embedding_size': [embedding_size], 'seq_len': [seq_len],
       'accuracy': [model_metrics[1]], 'precision': [model_metrics[2]], 'recall': [model_metrics[3]]}
metrics_df = pd.concat([metrics_df, pd.DataFrame(data=data)])
print('----------------------Model trained--------------------------------')

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 11, 20)            520       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 11, 20)            820       
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 11, 20)            820       
_________________________________________________________________
flatten (Flatten)            (None, 220)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 27)                5967      
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 112       
Total params: 8,239
Trainable params: 8,239
Non-trainable params: 0
____________________________________________________

Epoch 45/300
1416/1416 - 20s - loss: 1.1821 - accuracy: 0.4757 - precision_9: 0.6393 - recall_9: 0.1971
Epoch 46/300
1416/1416 - 20s - loss: 1.1817 - accuracy: 0.4759 - precision_9: 0.6390 - recall_9: 0.1974
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_11/assets
Epoch 47/300
1416/1416 - 20s - loss: 1.1796 - accuracy: 0.4766 - precision_9: 0.6409 - recall_9: 0.1990
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_11/assets
Epoch 48/300
1416/1416 - 20s - loss: 1.1803 - accuracy: 0.4769 - precision_9: 0.6408 - recall_9: 0.2019
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_11/assets
Epoch 49/300
1416/1416 - 20s - loss: 1.1777 - accuracy: 0.4781 - precision_9: 0.6442 - recall_9: 0.2045
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_11/assets
Epoch 50/300
1416/1416 - 20s - loss: 1.1780 - accuracy: 0.4766 - precision_9: 0.6436 - recall_9: 0.2045
Epoch 51/300
1416/1416 - 20s - loss: 1.1768 - accuracy: 

Epoch 107/300
1416/1416 - 18s - loss: 1.1458 - accuracy: 0.4889 - precision_9: 0.6613 - recall_9: 0.2306
Epoch 108/300
1416/1416 - 18s - loss: 1.1478 - accuracy: 0.4895 - precision_9: 0.6611 - recall_9: 0.2278
Epoch 109/300
1416/1416 - 18s - loss: 1.1446 - accuracy: 0.4918 - precision_9: 0.6652 - recall_9: 0.2286
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_11/assets
Epoch 110/300
1416/1416 - 18s - loss: 1.1454 - accuracy: 0.4908 - precision_9: 0.6649 - recall_9: 0.2303
Epoch 111/300
1416/1416 - 18s - loss: 1.1421 - accuracy: 0.4896 - precision_9: 0.6639 - recall_9: 0.2327
Epoch 112/300
1416/1416 - 18s - loss: 1.1471 - accuracy: 0.4888 - precision_9: 0.6691 - recall_9: 0.2330
Epoch 113/300
1416/1416 - 18s - loss: 1.1449 - accuracy: 0.4913 - precision_9: 0.6642 - recall_9: 0.2323
Epoch 114/300
1416/1416 - 18s - loss: 1.1444 - accuracy: 0.4890 - precision_9: 0.6633 - recall_9: 0.2318
Epoch 115/300
1416/1416 - 18s - loss: 1.1437 - accuracy: 0.4920 - precision_9: 0

Epoch 177/300
1416/1416 - 18s - loss: 1.1317 - accuracy: 0.4979 - precision_9: 0.6680 - recall_9: 0.2460
Epoch 178/300
1416/1416 - 18s - loss: 1.1291 - accuracy: 0.4982 - precision_9: 0.6720 - recall_9: 0.2432
Epoch 179/300
1416/1416 - 18s - loss: 1.1268 - accuracy: 0.4993 - precision_9: 0.6721 - recall_9: 0.2449
Epoch 180/300
1416/1416 - 18s - loss: 1.1309 - accuracy: 0.4969 - precision_9: 0.6725 - recall_9: 0.2435
Epoch 181/300
1416/1416 - 18s - loss: 1.1279 - accuracy: 0.4982 - precision_9: 0.6752 - recall_9: 0.2454
Epoch 182/300
1416/1416 - 18s - loss: 1.1265 - accuracy: 0.4994 - precision_9: 0.6743 - recall_9: 0.2449
Epoch 183/300
1416/1416 - 18s - loss: 1.1278 - accuracy: 0.4991 - precision_9: 0.6729 - recall_9: 0.2473
Epoch 184/300
1416/1416 - 18s - loss: 1.1261 - accuracy: 0.4992 - precision_9: 0.6776 - recall_9: 0.2475
Epoch 185/300
1416/1416 - 18s - loss: 1.1262 - accuracy: 0.4984 - precision_9: 0.6779 - recall_9: 0.2459
Epoch 186/300
1416/1416 - 18s - loss: 1.1277 - accuracy

Epoch 251/300
1416/1416 - 18s - loss: 1.1179 - accuracy: 0.5024 - precision_9: 0.6802 - recall_9: 0.2548
Epoch 252/300
1416/1416 - 18s - loss: 1.1175 - accuracy: 0.5024 - precision_9: 0.6782 - recall_9: 0.2557
Epoch 253/300
1416/1416 - 18s - loss: 1.1133 - accuracy: 0.5044 - precision_9: 0.6810 - recall_9: 0.2554
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_11/assets
Epoch 254/300
1416/1416 - 18s - loss: 1.1158 - accuracy: 0.5020 - precision_9: 0.6790 - recall_9: 0.2538
Epoch 255/300
1416/1416 - 18s - loss: 1.1192 - accuracy: 0.5025 - precision_9: 0.6785 - recall_9: 0.2552
Epoch 256/300
1416/1416 - 18s - loss: 1.1182 - accuracy: 0.5014 - precision_9: 0.6802 - recall_9: 0.2543
Epoch 257/300
1416/1416 - 18s - loss: 1.1150 - accuracy: 0.5022 - precision_9: 0.6814 - recall_9: 0.2542
Epoch 258/300
1416/1416 - 18s - loss: 1.1140 - accuracy: 0.5044 - precision_9: 0.6845 - recall_9: 0.2576
Epoch 259/300
1416/1416 - 18s - loss: 1.1179 - accuracy: 0.5016 - precision_9: 0

In [14]:
epochs = 300
embedding_size = 25
seq_len = 11
X_train, X_val, y_train, y_val = prepare_data(simpson_model_25_path, simpson_prefix, seq_len, embedding_size)
callback = create_checkpoint_callback('./resources/checkpoints/baseline_' + 
                                      str(embedding_size) + '_' +str(seq_len))
model = create_RNN_model(embedding_size, seq_len, 4)
model.fit(X_train, y_train, epochs = epochs, verbose=2, callbacks=[callback])
model_metrics = model.evaluate(x=X_val, y=y_val)
data = {'model_name': 'simpleRNN', 'embedding_size': [embedding_size], 'seq_len': [seq_len],
       'accuracy': [model_metrics[1]], 'precision': [model_metrics[2]], 'recall': [model_metrics[3]]}
metrics_df = pd.concat([metrics_df, pd.DataFrame(data=data)])
print('----------------------Model trained--------------------------------')

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 11, 20)            920       
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 11, 20)            820       
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 11, 20)            820       
_________________________________________________________________
flatten_1 (Flatten)          (None, 220)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 137)               30277     
_________________________________________________________________
dense_21 (Dense)             (None, 4)                 552       
Total params: 33,389
Trainable params: 33,389
Non-trainable params: 0
_________________________________________________

Epoch 39/300
1416/1416 - 18s - loss: 0.8269 - accuracy: 0.6535 - precision_10: 0.7524 - recall_10: 0.5305
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 40/300
1416/1416 - 18s - loss: 0.8236 - accuracy: 0.6549 - precision_10: 0.7540 - recall_10: 0.5338
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 41/300
1416/1416 - 18s - loss: 0.8165 - accuracy: 0.6572 - precision_10: 0.7557 - recall_10: 0.5352
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 42/300
1416/1416 - 18s - loss: 0.8091 - accuracy: 0.6627 - precision_10: 0.7589 - recall_10: 0.5432
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 43/300
1416/1416 - 18s - loss: 0.8068 - accuracy: 0.6619 - precision_10: 0.7575 - recall_10: 0.5437
Epoch 44/300
1416/1416 - 18s - loss: 0.7994 - accuracy: 0.6650 - precision_10: 0.7603 - recall_10: 0.5496
INFO:tensorflow:Assets written to: ./res

Epoch 90/300
1416/1416 - 18s - loss: 0.6500 - accuracy: 0.7301 - precision_10: 0.8068 - recall_10: 0.6474
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 91/300
1416/1416 - 18s - loss: 0.6474 - accuracy: 0.7308 - precision_10: 0.8076 - recall_10: 0.6504
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 92/300
1416/1416 - 18s - loss: 0.6473 - accuracy: 0.7311 - precision_10: 0.8091 - recall_10: 0.6496
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 93/300
1416/1416 - 18s - loss: 0.6437 - accuracy: 0.7326 - precision_10: 0.8109 - recall_10: 0.6515
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 94/300
1416/1416 - 18s - loss: 0.6420 - accuracy: 0.7325 - precision_10: 0.8112 - recall_10: 0.6518
Epoch 95/300
1416/1416 - 18s - loss: 0.6422 - accuracy: 0.7325 - precision_10: 0.8070 - recall_10: 0.6520
Epoch 96/300
1416/1416 - 18s - loss: 0.6

INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 147/300
1416/1416 - 18s - loss: 0.5758 - accuracy: 0.7609 - precision_10: 0.8319 - recall_10: 0.6902
Epoch 148/300
1416/1416 - 18s - loss: 0.5751 - accuracy: 0.7613 - precision_10: 0.8347 - recall_10: 0.6915
Epoch 149/300
1416/1416 - 18s - loss: 0.5711 - accuracy: 0.7627 - precision_10: 0.8310 - recall_10: 0.6934
Epoch 150/300
1416/1416 - 18s - loss: 0.5748 - accuracy: 0.7635 - precision_10: 0.8322 - recall_10: 0.6942
Epoch 151/300
1416/1416 - 18s - loss: 0.5671 - accuracy: 0.7635 - precision_10: 0.8344 - recall_10: 0.6968
Epoch 152/300
1416/1416 - 18s - loss: 0.5704 - accuracy: 0.7622 - precision_10: 0.8308 - recall_10: 0.6941
Epoch 153/300
1416/1416 - 18s - loss: 0.5719 - accuracy: 0.7622 - precision_10: 0.8330 - recall_10: 0.6942
Epoch 154/300
1416/1416 - 18s - loss: 0.5700 - accuracy: 0.7647 - precision_10: 0.8351 - recall_10: 0.6970
INFO:tensorflow:Assets written to: ./resources/checkpoints/base

Epoch 213/300
1416/1416 - 18s - loss: 0.5334 - accuracy: 0.7793 - precision_10: 0.8481 - recall_10: 0.7178
Epoch 214/300
1416/1416 - 18s - loss: 0.5293 - accuracy: 0.7818 - precision_10: 0.8501 - recall_10: 0.7183
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 215/300
1416/1416 - 18s - loss: 0.5202 - accuracy: 0.7855 - precision_10: 0.8535 - recall_10: 0.7222
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_25_11/assets
Epoch 216/300
1416/1416 - 18s - loss: 0.5301 - accuracy: 0.7813 - precision_10: 0.8466 - recall_10: 0.7187
Epoch 217/300
1416/1416 - 18s - loss: 0.5304 - accuracy: 0.7816 - precision_10: 0.8492 - recall_10: 0.7204
Epoch 218/300
1416/1416 - 18s - loss: 0.5259 - accuracy: 0.7832 - precision_10: 0.8505 - recall_10: 0.7222
Epoch 219/300
1416/1416 - 18s - loss: 0.5247 - accuracy: 0.7814 - precision_10: 0.8481 - recall_10: 0.7196
Epoch 220/300
1416/1416 - 18s - loss: 0.5295 - accuracy: 0.7812 - precision_10: 0.8474 - 

Epoch 282/300
1416/1416 - 19s - loss: 0.5036 - accuracy: 0.7921 - precision_10: 0.8578 - recall_10: 0.7343
Epoch 283/300
1416/1416 - 19s - loss: 0.5001 - accuracy: 0.7937 - precision_10: 0.8599 - recall_10: 0.7356
Epoch 284/300
1416/1416 - 18s - loss: 0.4953 - accuracy: 0.7932 - precision_10: 0.8589 - recall_10: 0.7357
Epoch 285/300
1416/1416 - 18s - loss: 0.5085 - accuracy: 0.7892 - precision_10: 0.8541 - recall_10: 0.7297
Epoch 286/300
1416/1416 - 18s - loss: 0.5001 - accuracy: 0.7935 - precision_10: 0.8583 - recall_10: 0.7353
Epoch 287/300
1416/1416 - 18s - loss: 0.4983 - accuracy: 0.7946 - precision_10: 0.8595 - recall_10: 0.7359
Epoch 288/300
1416/1416 - 18s - loss: 0.5045 - accuracy: 0.7924 - precision_10: 0.8572 - recall_10: 0.7347
Epoch 289/300
1416/1416 - 18s - loss: 0.4982 - accuracy: 0.7945 - precision_10: 0.8593 - recall_10: 0.7349
Epoch 290/300
1416/1416 - 18s - loss: 0.4942 - accuracy: 0.7957 - precision_10: 0.8603 - recall_10: 0.7387
INFO:tensorflow:Assets written to: ./

In [16]:
metrics_df.to_csv('./resources/metrics_df.csv')

In [15]:
epochs = 300
embedding_size = 125
seq_len = 11
X_train, X_val, y_train, y_val = prepare_data(simpson_model_125_path, simpson_prefix, seq_len, embedding_size)
callback = create_checkpoint_callback('./resources/checkpoints/baseline_' + 
                                      str(embedding_size) + '_' +str(seq_len))
model = create_RNN_model(embedding_size, seq_len, 4)
model.fit(X_train, y_train, epochs = epochs, verbose=2, callbacks=[callback])
model_metrics = model.evaluate(x=X_val, y=y_val)
data = {'model_name': 'simpleRNN', 'embedding_size': [embedding_size], 'seq_len': [seq_len],
       'accuracy': [model_metrics[1]], 'precision': [model_metrics[2]], 'recall': [model_metrics[3]]}
metrics_df = pd.concat([metrics_df, pd.DataFrame(data=data)])
print('----------------------Model trained--------------------------------')

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_6 (SimpleRNN)     (None, 11, 20)            2920      
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 11, 20)            820       
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 11, 20)            820       
_________________________________________________________________
flatten_2 (Flatten)          (None, 220)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 687)               151827    
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 2752      
Total params: 159,139
Trainable params: 159,139
Non-trainable params: 0
_______________________________________________

KeyboardInterrupt: 

## LSTM

In [ ]:
def create_RNN_model(embedding_size, seq_len, classes):
    model = keras.Sequential([
        layers.LSTM(units = 20, return_sequences=True, input_shape=(seq_len, embedding_size)),
        layers.LSTM(units = 20, return_sequences=True),
        layers.LSTM(units = 20, return_sequences=True),
        layers.Flatten(),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [ ]:
epochs = 300
embedding_size = 5
seq_len = 11
X_train, X_val, y_train, y_val = prepare_data(simpson_model_5_path, simpson_prefix, seq_len, embedding_size)
callback = create_checkpoint_callback('./resources/checkpoints/baseline_' + 
                                      str(embedding_size) + '_' +str(seq_len))
model = create_RNN_model(embedding_size, seq_len, 4)
model.fit(X_train, y_train, epochs = epochs, verbose=2, callbacks=[callback])
model_metrics = model.evaluate(x=X_val, y=y_val)
data = {'model_name': 'LSTM', 'embedding_size': [embedding_size], 'seq_len': [seq_len],
       'accuracy': [model_metrics[1]], 'precision': [model_metrics[2]], 'recall': [model_metrics[3]]}
metrics_df = pd.concat([metrics_df, pd.DataFrame(data=data)])
print('----------------------Model trained--------------------------------')

In [ ]:
embedding_size = 25
seq_len = 11
X_train, X_val, y_train, y_val = prepare_data(simpson_model_25_path, simpson_prefix, seq_len, embedding_size)
callback = create_checkpoint_callback('./resources/checkpoints/baseline_' + 
                                      str(embedding_size) + '_' +str(seq_len))
model = create_RNN_model(embedding_size, seq_len, 4)
model.fit(X_train, y_train, epochs = epochs, verbose=2, callbacks=[callback])
model_metrics = model.evaluate(x=X_val, y=y_val)
data = {'model_name': 'LSTM', 'embedding_size': [embedding_size], 'seq_len': [seq_len],
       'accuracy': [model_metrics[1]], 'precision': [model_metrics[2]], 'recall': [model_metrics[3]]}
metrics_df = pd.concat([metrics_df, pd.DataFrame(data=data)])
print('----------------------Model trained--------------------------------')

In [ ]:
embedding_size = 125
seq_len = 11
X_train, X_val, y_train, y_val = prepare_data(simpson_model_125_path, simpson_prefix, seq_len, embedding_size)
callback = create_checkpoint_callback('./resources/checkpoints/baseline_' + 
                                      str(embedding_size) + '_' +str(seq_len))
model = create_RNN_model(embedding_size, seq_len, 4)
model.fit(X_train, y_train, epochs = epochs, verbose=2, callbacks=[callback])
model_metrics = model.evaluate(x=X_val, y=y_val)
data = {'model_name': 'LSTM', 'embedding_size': [embedding_size], 'seq_len': [seq_len],
       'accuracy': [model_metrics[1]], 'precision': [model_metrics[2]], 'recall': [model_metrics[3]]}
metrics_df = pd.concat([metrics_df, pd.DataFrame(data=data)])
print('----------------------Model trained--------------------------------')